[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1DFPverSl5ADn2czBmxfDrJdChgJ93LGi?usp=sharing)

In [ ]:
!nvidia-smi

Fri May 26 00:01:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.57.02    Driver Version: 516.93       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:0B:00.0 Off |                  N/A |
| 35%   57C    P0   111W / 350W |     14MiB / 12288MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 6000     On   | 00000000:43:00.0 Off |                  Off |
| 42%   

In [ ]:
from datasets import load_dataset

data = load_dataset('AnyaSchen/image2poetry_ru')

Found cached dataset parquet (/home/revolt/.cache/huggingface/datasets/AnyaSchen___parquet/AnyaSchen--image2poetry_ru-bd53c8b353e828ac/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████████████████████████████████████████████| 1/1 [00:00<00:00, 55.98it/s]


In [ ]:
import torch
device = torch.device('cuda:0')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from PIL import Image
import requests
from transformers import AutoTokenizer, CLIPProcessor, VisionEncoderDecoderModel, ViTImageProcessor

def generate_poetry(fine_tuned_model, image, tokenizer, author):
    # Preprocess the image using the CLIP processor
    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)
    
    # Encode author's name and prepare as input to the decoder
    author_input = f"<bos> {author} <sep>"
    decoder_input_ids = tokenizer.encode(author_input, return_tensors="pt").to(device)

    # Generate the poetry with the fine-tuned VisionEncoderDecoder model
    generated_tokens = fine_tuned_model.generate(
        pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length=300,
        num_beams=3,
        top_p=0.8,
        temperature=2.0,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Decode the generated tokens
    generated_poetry = tokenizer.decode(generated_tokens[0], skip_special_tokens=True).replace('�', '')
    generated_poetry = generated_poetry.split(f'{author}')[-1]
    return generated_poetry

/home/revolt/anaconda3/envs/poetry_gpt3_large/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from tqdm import tqdm

In [ ]:
import pickle

def upload_generated_poetry(data: dict, file_name):
  with open(file_name, 'wb') as f:
    pickle.dump(data, f)

# generate poetry by fune-tuned vit-rugpt3 model

In [ ]:
path = 'AnyaSchen/vit-rugpt3-large-poetry-ft'
fine_tuned_model = VisionEncoderDecoderModel.from_pretrained(path).to(device)
feature_extractor = ViTImageProcessor.from_pretrained(path)

tokenizer = AutoTokenizer.from_pretrained(path)

poetry_all_poets = {author:[] for author in set(data['train']['author'])}
for i in tqdm(range(data['train'].num_rows)):
  author = data['train'][i]['author']
  poetry_all_poets[author].append(generate_poetry(fine_tuned_model, data['train'][i]['image'], tokenizer, author))

upload_generated_poetry(poetry_all_poets, f'./vit_rugpt2_ft.pkl')
del fine_tuned_model

  0%|                                                       | 0/7755 [00:00<?, ?it/s]/home/revolt/anaconda3/envs/poetry_gpt3_large/lib/python3.9/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
 10%|████▏                                    | 797/7755 [1:00:45<7:09:26,  3.70s/it]

# generate poetry by vit-rugpt3-medium-poet model

In [ ]:
info_author = {
    'Маяковский':'mayak',
    'Тютчев':'tyutchev',
    'Блок' : 'blok',
    'Пушкин': 'pushkin',
    'Есенин': 'esenin'
}

In [ ]:
for author in tqdm(info_author):
  path = f'{info_author[author]}/'
# Load the fine-tuned model
  fine_tuned_model = VisionEncoderDecoderModel.from_pretrained(f'./{path}model')
  feature_extractor = ViTImageProcessor.from_pretrained(f'./{path}vit_feature_extractor')

  # Load a GPT tokenizer for the Russian language
  tokenizer = AutoTokenizer.from_pretrained(f'./{path}tokenizer')
  fine_tuned_model.to(device)

  poetry_list = []
  for i in tqdm(range(data['train'].num_rows)):
    author_curr = data['train'][i]['author']
    if author_curr == author:
      poetry_list.append(generate_poetry(fine_tuned_model, data['train'][i]['image'], tokenizer, ''))

  upload_generated_poetry({author: poetry_list}, f'./{path}generated_image2poetry_vit_rugpt2_{info_author[author]}.pkl')
  del fine_tuned_model

Выходные данные были обрезаны до нескольких последних строк (5000).
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [7:57:18<00:00, 9546.02s/it]
